In [6]:
%pip install requests
%pip install pandas

import requests
import json
import pandas as pd
import hashlib
import random


Note: you may need to restart the kernel to use updated packages.


In [7]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1727889337.4257207,
   'transactions': []},
  {'index': 2,
   'nonce': 4199478024198298982,
   'previous_hash': '8f11c7800786161885d6f3b8481e636fd23e2c540d94b51168746cb2081e7bef',
   'timestamp': 1727889402.5228713,
   'transactions': [{'amount': 3,
     'recipient': 'test_to',
     'sender': 'test_from',
     'timestamp': 1727889400.3738093},
    {'amount': 0.1,
     'recipient': 'node_5000',
     'sender': 'master',
     'timestamp': 1727889402.5228713}]}],
 'length': 2}

In [8]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
data = {
    "sender": "test_from",
    "recipient": "test_to",
    "amount": 3
}
res = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data))
json.loads(res.content)

{'message': 'Transaction will be added to Block {3}'}

In [9]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine", headers=headers)
print(res)
print(res.text)

<Response [200]>
{"index":3,"message":"new block found!","nonce":11042817314664589791,"previous_hash":"31c961eb02fbf1385b2ca46bae5b33ed7c312c8c20d76460f4ed2560f6566465","transactions":[{"amount":3,"recipient":"test_to","sender":"test_from","timestamp":1727889413.2469058},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1727889415.38105}]}



In [10]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
status_json = json.loads(res.content)

tx_amount_l = []
tx_sender_l = []
tx_reciv_l = []
tx_time_l = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])

df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l
df_tx['sender'] = tx_sender_l
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l
df_tx

,timestamp,sender,recipient,amount
0,1.727889e+09,test_from,test_to,3.0
1,1.727889e+09,master,node_5000,0.1
2,1.727889e+09,test_from,test_to,3.0
3,1.727889e+09,master,node_5000,0.1


In [11]:
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user', 'sended_amount']
df_recived = pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_recived.columns = ['user', 'recived_amount']

df_status = pd.merge(df_sended, df_recived, on='user', how='outer').fillna(0)
df_status['balance'] = df_status['recived_amount'] - df_status['sended_amount']
df_status

,user,sended_amount,recived_amount,balance
0,master,0.2,0.0,-0.2
1,node_5000,0.0,0.2,0.2
2,test_from,6.0,0.0,-6.0
3,test_to,0.0,6.0,6.0
